In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
dl_train=pd.read_csv('./train.csv')
dl_train.head()

,Id,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,required_doughnuts_comsumption,fraudulent
0,0,5737,Jr. Business Analyst & Quality Analyst (entry...,"US, NJ, PISCATAWAY",NaN,NaN,NaN,"Duration: Full time / W2Location: Piscataway,N...",What we require:-- Masters degree in Computers...,NaN,0,0,0,Full-time,Entry level,Master's Degree,Financial Services,Finance,-0.410911,0
1,1,7107,English Teacher Abroad,"US, PA, Scranton",NaN,NaN,We help teachers get safe &amp; secure jobs ab...,"Play with kids, get paid for it Love travel? J...",University degree required. TEFL / TESOL / CEL...,See job description,0,1,1,Contract,NaN,Bachelor's Degree,Education Management,NaN,-0.163785,0
2,2,11979,SQL Server Database Developer Job opportunity ...,"US, IL, Barrington",NaN,90000-100000,We are an innovative personnel-sourcing firm w...,Position : SQL Server Database DeveloperJob Lo...,Position : SQL Server Database DeveloperJob Lo...,Benefits - FullBonus Eligible - Yes,0,0,0,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,-0.244369,0
3,3,9375,Legal Analyst - 12 Month FTC,"GB, LND, London",Legal,NaN,MarketInvoice is one of the most high-profile ...,DescriptionOur mission at MarketInvoice is to ...,Duties and ResponsibilitiesReviewing contractu...,Competitive salaryPrivate HealthcareHalf price...,0,1,0,Full-time,Associate,Professional,Financial Services,Legal,-0.456807,0
4,4,1301,Part-Time Finance Assistant,"GB, LND,",NaN,NaN,NaN,Salary:£9 - £10 per hour We are currently goin...,"Your role will be a varied, interesting and in...",Salary:£9 - £10 per hour,0,0,0,Part-time,NaN,NaN,Accounting,NaN,-0.217880,0


In [3]:
dl_test=pd.read_csv('./test.csv')
dl_test.head()

,Id,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,doughnuts_comsumption
0,0,14155,Home Based Typist/ Data Entry Clerk,"ZA, WC, Oudtshoorn",NaN,NaN,NaN,We have several openings available in this are...,Applicants should have a computer with interne...,"Averaging R2,500 per week depending on if you ...",0,0,0,NaN,NaN,NaN,NaN,NaN,1.763896
1,1,6567,Call Center Representative I,"US, TX, AUSTIN",CALL CENTER,28000-32000,As the industry’s largest supply contracting c...,The Call Center Representative I will provide ...,1-2 years customer serviceExcellent communicat...,competitive salaries and comprehensive benefit...,0,1,1,Full-time,Associate,High School or equivalent,Consumer Services,Customer Service,0.285890
2,2,5930,Chief Accountant,"RO, B, Bucharrest",Accounting,NaN,At The Practice Group (TPG) we provide clients...,TPG is searching a Chief Accountant for one of...,Good knowledge of SAPFluency in EnglishUnivers...,NaN,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Accounting,Accounting/Auditing,-0.700898
3,3,11085,Office Services Associate,"US, CA, San Jose",NaN,NaN,"Novitex Enterprise Solutions, formerly Pitney ...",The Office Services Associate will be based in...,Required Qualifications: Ability to work with ...,NaN,0,1,0,Full-time,Associate,High School or equivalent,Banking,Customer Service,-2.235754
4,4,8381,Need Oracle Fusion HCM Resource,"US, OH, North Canton",NaN,NaN,Infolob Solutions is a leader in Managed IT se...,Oracle Fusion HCM ResourceLocation - North Can...,1. Minimum of one project (prefer 2 or mor...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,-1.154964


In [4]:
def loc_split(df):
    df['location'] = df['location'].replace(np.NAN, 'nan')
    df['country'] = np.NAN
    df['region'] = np.NAN
    df['city'] = np.NAN
    
    for i in range(df.shape[0]):
        df['location'].iloc[i] = df['location'].iloc[i].replace(" ", "").lower()
        chunk = df['location'].iloc[i].split(',')
        for j in range(len(chunk)):
            if j == 0:
                df['country'].iloc[i] = chunk[0]
            elif j == 1:
                df['region'].iloc[i] = chunk[1]
            elif j == 2:
                df['city'].iloc[i] = chunk[2]
            
    df.drop('location', axis = 1, inplace = True)
    
    # Replace np.NAN as a new category
    for feat in ['country', 'region', 'city']:
        df[feat] = df[feat].replace(np.NAN, 'nan')
    
    return df


In [5]:
# Removing features
def feat_rm(df, feats):
    for feat in feats:
        df.drop(feat, axis = 1, inplace = True)
    
    return df
    

# Scaling features
def feat_scaled(df, feat):
    feat_scaled = df[feat]
    
    feat_scaled = (feat_scaled - feat_scaled.mean()) / feat_scaled.std()
    
    df[feat] = feat_scaled
    
    return df



In [6]:
def fill_na(df, feats):
    for feat in feats:
        df[feat] = df[feat].replace(np.NAN, 'nan')
    return df


In [7]:
dl_train2 = loc_split(dl_train)

/Users/flaviaferrusmarimon/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [8]:
dl_train2

,Id,job_id,title,department,salary_range,company_profile,description,requirements,benefits,telecommuting,...,employment_type,required_experience,required_education,industry,function,required_doughnuts_comsumption,fraudulent,country,region,city
0,0,5737,Jr. Business Analyst & Quality Analyst (entry...,NaN,NaN,NaN,"Duration: Full time / W2Location: Piscataway,N...",What we require:-- Masters degree in Computers...,NaN,0,...,Full-time,Entry level,Master's Degree,Financial Services,Finance,-0.410911,0,us,nj,piscataway
1,1,7107,English Teacher Abroad,NaN,NaN,We help teachers get safe &amp; secure jobs ab...,"Play with kids, get paid for it Love travel? J...",University degree required. TEFL / TESOL / CEL...,See job description,0,...,Contract,NaN,Bachelor's Degree,Education Management,NaN,-0.163785,0,us,pa,scranton
2,2,11979,SQL Server Database Developer Job opportunity ...,NaN,90000-100000,We are an innovative personnel-sourcing firm w...,Position : SQL Server Database DeveloperJob Lo...,Position : SQL Server Database DeveloperJob Lo...,Benefits - FullBonus Eligible - Yes,0,...,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,-0.244369,0,us,il,barrington
3,3,9375,Legal Analyst - 12 Month FTC,Legal,NaN,MarketInvoice is one of the most high-profile ...,DescriptionOur mission at MarketInvoice is to ...,Duties and ResponsibilitiesReviewing contractu...,Competitive salaryPrivate HealthcareHalf price...,0,...,Full-time,Associate,Professional,Financial Services,Legal,-0.456807,0,gb,lnd,london
4,4,1301,Part-Time Finance Assistant,NaN,NaN,NaN,Salary:£9 - £10 per hour We are currently goin...,"Your role will be a varied, interesting and in...",Salary:£9 - £10 per hour,0,...,Part-time,NaN,NaN,Accounting,NaN,-0.217880,0,gb,lnd,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,10723,11225,Production Artist,Design,NaN,"We're an emerging technology agency, and we bu...",Your hand develops our complete digital visual...,Requirements/Qualifications:1+ years of intera...,Why work for Mutual Mobile? We craft beautiful...,0,...,Full-time,Associate,Bachelor's Degree,Computer Software,Design,0.208937,0,us,tx,austin
10724,10724,7567,iOS Developer,NaN,NaN,BlueLine Grid is transforming government commu...,BlueLine Grid is transforming government commu...,"BS/MS degree in Computer Science, Engineering ...","Full health, vision, dental benefitsNo vacatio...",0,...,Full-time,Mid-Senior level,Unspecified,NaN,Information Technology,-0.089956,0,us,,newyork
10725,10725,9098,Graduates: English Teacher Abroad,NaN,NaN,We help teachers get safe &amp; secure jobs ab...,"Play with kids, get paid for it :-)Love travel...",University degree required. TEFL / TESOL / CEL...,See job description,0,...,Contract,NaN,Bachelor's Degree,Education Management,NaN,0.335414,0,us,ca,fremont
10726,10726,4821,Senior IT Security Lead (SIEM / IDS),NaN,NaN,Roland &amp; Associates is a Corporate Recruit...,You will own the development of operational pl...,Job Role : Senior IT Security Lead (SIEM / ID...,Salary: Excellent package with full benefits a...,0,...,Full-time,Mid-Senior level,Associate Degree,Electrical/Electronic Manufacturing,Information Technology,-0.262851,0,us,co,coloradosprings


In [9]:
dl_train2.columns

Index(['Id', 'job_id', 'title', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'required_doughnuts_comsumption', 'fraudulent', 'country', 'region',
       'city'],
      dtype='object')

In [19]:
dl_train_f= dl_train2[dl_train2['fraudulent']==1]
dl_train_t= dl_train2[dl_train2['fraudulent']==0]
dl_train_f.shape[0]

494

In [11]:
print(pd.unique(dl_train_f['country']))
print(pd.unique(dl_train_t['country']))

['au' 'us' 'my' 'gb' 'qa' 'ca' 'nan' 'in' 'bh' 'ph' 'id' 'tw' 'pl']
['us' 'gb' 'be' 'ca' 'ar' 'dk' 'gr' 'de' 'es' 'pl' 'ch' 'ph' 'ae' 'nan'
 'au' 'ie' 'in' 'sg' 'nl' 'pa' 'ee' 'nz' 'ro' 'ru' 'qa' 'tn' 'hk' 'bg'
 'fr' 'eg' 'hu' 'ua' 'se' 'fi' 'il' 'pk' 'mx' 'cn' 'at' 'rs' 'ni' 'by'
 'br' 'kz' 'mu' 'pt' 'it' 'id' 'cz' 'sa' 'ng' 'za' 'tr' 'jp' 'zm' 'lv'
 'mt' 'lt' 'ke' 'hr' 'my' 'kr' 'iq' 'vn' 'no' 'ug' 'cy' 'th' 'al' 'is'
 'lu' 'am' 'sd' 'gh' 'bh' 'tt' 'si' 'lk' 'bd' 'vi' 'tw' 'kw' 'pe' 'sk']


In [17]:
print(len(dl_train_f[dl_train_f['country'] == 'nan']))
print(len(dl_train_t[dl_train_t['country'] == 'nan']))

TypeError: 'tuple' object is not callable

In [11]:
dl_test2 = loc_split(dl_test)

/Users/flaviaferrusmarimon/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [12]:
dl_test2

,Id,job_id,title,department,salary_range,company_profile,description,requirements,benefits,telecommuting,...,has_questions,employment_type,required_experience,required_education,industry,function,doughnuts_comsumption,country,region,city
0,0,14155,Home Based Typist/ Data Entry Clerk,NaN,NaN,NaN,We have several openings available in this are...,Applicants should have a computer with interne...,"Averaging R2,500 per week depending on if you ...",0,...,0,NaN,NaN,NaN,NaN,NaN,1.763896,za,wc,oudtshoorn
1,1,6567,Call Center Representative I,CALL CENTER,28000-32000,As the industry’s largest supply contracting c...,The Call Center Representative I will provide ...,1-2 years customer serviceExcellent communicat...,competitive salaries and comprehensive benefit...,0,...,1,Full-time,Associate,High School or equivalent,Consumer Services,Customer Service,0.285890,us,tx,austin
2,2,5930,Chief Accountant,Accounting,NaN,At The Practice Group (TPG) we provide clients...,TPG is searching a Chief Accountant for one of...,Good knowledge of SAPFluency in EnglishUnivers...,NaN,0,...,1,Full-time,Mid-Senior level,Bachelor's Degree,Accounting,Accounting/Auditing,-0.700898,ro,b,bucharrest
3,3,11085,Office Services Associate,NaN,NaN,"Novitex Enterprise Solutions, formerly Pitney ...",The Office Services Associate will be based in...,Required Qualifications: Ability to work with ...,NaN,0,...,0,Full-time,Associate,High School or equivalent,Banking,Customer Service,-2.235754,us,ca,sanjose
4,4,8381,Need Oracle Fusion HCM Resource,NaN,NaN,Infolob Solutions is a leader in Managed IT se...,Oracle Fusion HCM ResourceLocation - North Can...,1. Minimum of one project (prefer 2 or mor...,NaN,0,...,0,NaN,NaN,NaN,NaN,NaN,-1.154964,us,oh,northcanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7147,7147,10956,Customer Service Associate,NaN,NaN,We take the hassle out of finding and booking ...,About UsWe take the hassle out of finding and ...,About YouYou must have previous customer servi...,BenefitsSalary is £16-20k depending on experie...,0,...,0,Full-time,NaN,NaN,Consumer Goods,Customer Service,0.887867,gb,lnd,london
7148,7148,17290,QA Engineer,NaN,NaN,Xplenty is all about making complex stuff look...,"As a QA engineer, you will be responsible for ...",You have to love QA and feel a compulsion to t...,About Xplenty:Xplenty is all about making comp...,0,...,1,Full-time,NaN,NaN,NaN,NaN,-0.789032,il,,
7149,7149,5193,HGV Class 1 - Trunk Driver,NaN,NaN,With our head office in Radstock the Unique Gr...,We urgently require both Day and Night Trunker...,HGV/LGV Class 1 license.Digi Tacho.,Regular ongoing work.Various start times.Good ...,0,...,1,NaN,NaN,NaN,NaN,NaN,-2.359534,gb,,
7150,7150,12173,Firmware Engineer - C# Visual Basic ASM SQL - FL,NaN,NaN,We Provide Full Time Permanent Positions for m...,Experienced Firmware Engineer is required havi...,Experience in programming microcontrollersEmbe...,NaN,0,...,0,Full-time,NaN,NaN,NaN,NaN,-1.317668,us,fl,daytona
